# two-dimensional bending plate example
## Import Optimizer class

In [11]:
from anisogrid.tools import Optimizer
from fenics import *
from fenics_adjoint import *
import numpy as np


## Mesh generation

In [12]:
mesh = RectangleMesh(Point(0,0), Point(50,20), 50, 20)
N = mesh.num_vertices()

## Defin SubDomains for boundary conditions

In [14]:
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < 0.01 and on_boundary

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] > 45 and x[1] < 0.1 and on_boundary

class Loading(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] > 19.9 and x[0] < 5. and on_boundary

## Define displacement boundary conditions

In [15]:
displacement_boundaries_0 = [Left()]
displacement_boundaries_1 = [Right()]
applied_disp_0 = [Constant(0)]
applied_disp_1 = [Constant(0)]

## Define loading boundary conditions

In [16]:
loading_boundaries = [Loading()]
applied_loads = [Constant((0, -1))]

## Define material properties

In [17]:
material = {'E1': 3600, 'E2': 600, 'nu12':0.33, 'G12': 500}
path = 'results/example1/'
files = {'Displacement': XDMFFile('{}displacement.xdmf'.format(path)),
         'Stress': XDMFFile('{}stress.xdmf'.format(path)),
         'Strain': XDMFFile('{}strain.xdmf'.format(path)),
         'Orient': XDMFFile('{}orient.xdmf'.format(path)),
         'Orientpipe': File('{}orient.xml'.format(path)),
         'Denspipe': File('{}dens.xml'.format(path)),
         'Dens': XDMFFile('{}dens.xdmf'.format(path))
        }

## Initialize all design variables

In [24]:
z0 = np.ones(N)
e0 = np.zeros(N)
r0 = np.ones(N)*0.5
x0 = np.concatenate([z0, e0, r0])

## Optimizer setup

In [31]:
opt = Optimizer()
opt.set_mesh(mesh)
opt.set_bcs(displacement_boundaries_0, displacement_boundaries_1, applied_disp_0, applied_disp_1)
opt.set_loading(loading_boundaries, applied_loads)
opt.set_material(material)
opt.set_working_dir(files)
opt.set_target(0.9)
opt.initialize()
opt.run(x0)

0.17793090845932777
MMA dual converged in 6 iterations to g=0.177931:
    MMA y[0]=1e+40, gc[0]=0.1
0.17793090845932777
